In [49]:
import pandas as pd
dataFolder = '../Data/wikispeedia_paths-and-graph/'
articlesDf = pd.read_csv(dataFolder+ 'articles.tsv')
articlesDf = articlesDf.iloc[16:]
#articlesSeries  = pd.Series(articlesDf.index)
#articlesSeries
articlesDf.



NotImplementedError: initializing a Series from a MultiIndex is not supported